In [1]:
# Constants and Configuration Variables
DEBUG = False
TRAIN = False
TUNING = True
OVERWRITE_PROCESSED_DATA = True
N_TRIALS = 10
STATE = 42

gpu_switch = "OFF"
N_SPLITS = 7
N_TEST_SPLITS = 1
N_PURGE = 11
N_EMBARGO = 11


VERSION_NB = 11
EXPERIMENT_PURPOSE = "optiver_trading_at_the_close"


# RECORD ENSAMBLE MODEL
model_paths = [
    "s3://mlflow-v1/kaggle_optiver_trading_at_the_close/0f6ce4df6744464bbd4123566a088d27/artifacts/LGBMR_0_20231114_095007/model.pkl",
    "s3://mlflow-v1/kaggle_optiver_trading_at_the_close/47d294f60f9642248cb2d0fb8ccd9bf6/artifacts/LGBMR_0_20231114_100521/model.pkl",
    "s3://mlflow-v1/kaggle_optiver_trading_at_the_close/9431eeeeaf6b4690a6a11d9b9d0ad52a/artifacts/LGBMR_0_20231114_101915/model.pkl",
    "s3://mlflow-v1/kaggle_optiver_trading_at_the_close/e603fb30d1b04756b34e443325924da4/artifacts/LGBMR_0_20231114_103413/model.pkl",
    "s3://mlflow-v1/kaggle_optiver_trading_at_the_close/748ddac91acf4ead85cc0f567d053596/artifacts/LGBMR_0_20231114_104931/model.pkl",
    "s3://mlflow-v1/kaggle_optiver_trading_at_the_close/814b2dc76eb14d57a80a79e100c50c3a/artifacts/LGBMR_0_20231114_110439/model.pkl",
    "s3://mlflow-v1/kaggle_optiver_trading_at_the_close/5ab9ade2bcdb46e8bd86ae1e5f8f2df4/artifacts/LGBMR_0_20231114_111703/model.pkl"
]

# Define the model name for registration in MLflow

model_name = f"ensemble_model (7)"
folder_model = f'models-v8'
model_name_artifict = f"ensemble_model (7).pkl"

import joblib
model_prod = joblib.load('/kaggle/input/models-v8/ensemble_model (7).pkl')

In [2]:
model_prod.models[0].feature_name_

['stock_id',
 'seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'mid_price',
 'liquidity_imbalance',
 'matched_imbalance',
 'size_imbalance',
 'reference_price_far_price_imb',
 'reference_price_near_price_imb',
 'reference_price_ask_price_imb',
 'reference_price_bid_price_imb',
 'reference_price_wap_imb',
 'far_price_near_price_imb',
 'far_price_ask_price_imb',
 'far_price_bid_price_imb',
 'far_price_wap_imb',
 'near_price_ask_price_imb',
 'near_price_bid_price_imb',
 'near_price_wap_imb',
 'ask_price_bid_price_imb',
 'ask_price_wap_imb',
 'bid_price_wap_imb',
 'ask_price_bid_price_wap_imb2',
 'ask_price_bid_price_reference_price_imb2',
 'ask_price_wap_reference_price_imb2',
 'bid_price_wap_reference_price_imb2',
 'matched_size_bid_size_ask_size_imb2',
 'matched_size_bid_size_imbalance_size_imb2',
 'matched_size_ask_size_imbalance_size_imb2'

In [3]:
model_prod.models[0].feature_name_

['stock_id',
 'seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'mid_price',
 'liquidity_imbalance',
 'matched_imbalance',
 'size_imbalance',
 'reference_price_far_price_imb',
 'reference_price_near_price_imb',
 'reference_price_ask_price_imb',
 'reference_price_bid_price_imb',
 'reference_price_wap_imb',
 'far_price_near_price_imb',
 'far_price_ask_price_imb',
 'far_price_bid_price_imb',
 'far_price_wap_imb',
 'near_price_ask_price_imb',
 'near_price_bid_price_imb',
 'near_price_wap_imb',
 'ask_price_bid_price_imb',
 'ask_price_wap_imb',
 'bid_price_wap_imb',
 'ask_price_bid_price_wap_imb2',
 'ask_price_bid_price_reference_price_imb2',
 'ask_price_wap_reference_price_imb2',
 'bid_price_wap_reference_price_imb2',
 'matched_size_bid_size_ask_size_imb2',
 'matched_size_bid_size_imbalance_size_imb2',
 'matched_size_ask_size_imbalance_size_imb2'

In [4]:


# External general-purpose modules
import gc
import sys
import os
import itertools as itt
from itertools import combinations, product
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl

from pathlib import Path
import warnings
from numba import njit, prange
import boto3
from botocore.exceptions import NoCredentialsError
from dotenv import load_dotenv


# Setting pandas options and warning filters
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# Load environment variables
load_dotenv()
path_root_project = Path.cwd()
if path_root_project.name not in ["working", "content"]:
    path_root_project = Path(os.getenv("ROOT_PATH") or path_root_project)

    directories_to_add = ["utils", "feat_engineering", "validation"]
    for directory in directories_to_add:
        sys.path.append(str(path_root_project / "src" / directory))

        
from utils_training import create_model, experiments_data
from utils_data import load_config, load_dataset, reduce_mem_usage, PathManager
from utils_kaggle import (
    setup_kaggle,
    download_data,
    get_data,
    clean_directory_except_one,
)

from fe_optiver_trading_at_the_close import calculate_triplet_imbalance_numba, compute_triplet_imbalance, convert_weights_to_dict, global_stock_id_feats

pm = PathManager(path_root_project)

if TRAIN:
    if pm.path_root_project.name == "working":
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        aws_access_key_id = user_secrets.get_secret("AWS_ACCESS_KEY_ID")
        aws_region = user_secrets.get_secret("AWS_DEFAULT_REGION")
        aws_secret_access_key = user_secrets.get_secret("AWS_SECRET_ACCESS_KEY")
        s3_bucket_name = user_secrets.get_secret("S3_BUCKET")

        # Set AWS credentials in the environment variables
        os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
        os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key
        os.environ['AWS_DEFAULT_REGION'] = aws_region
    else:
        aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
        aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')


In [5]:
#clean_directory_except_one('/kaggle/working/','submission.csv')

In [6]:


# Conditional imports and settings based on TRAIN constant
if TRAIN:
    if pm.path_root_project.name == "working":
        !pip install loguru mlflow optuna > /dev/null

    

    from utils_mlflow import (
        get_experiments_df,
        delete_runs_and_artifacts,
        download_and_load_model,
        load_models_and_create_ensemble,
        save_and_register_model,
        log_model_parameters,
        get_or_create_experiment,
        experiments_data,
    )
    from utils_feat_importance import log_feature_importance, aggregate_feature_importance
    from model_validation import time_series_split

    # External Libraries
    import boto3
    from botocore.exceptions import NoCredentialsError
    from mlflow.exceptions import MlflowException
    import lightgbm as lgbm
    import mlflow
    import optuna
    from mlflow.tracking import MlflowClient
    from optuna.integration.mlflow import MLflowCallback
    from sklearn.model_selection import KFold
    from xgboost import XGBRegressor as XGBR
    from lightgbm import log_evaluation, early_stopping, LGBMRegressor as LGBMR



    # Auto-reload modules - Specific to Jupyter Notebooks
    %load_ext autoreload
    %autoreload 2

    mlflow.set_tracking_uri(pm.path_experiments_dir)
    client = MlflowClient()
    
    # Create an S3 client
    s3 = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
    )

In [7]:
if TRAIN:
    s3 = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
    )
    # Load the models and create an ensemble
    ensemble_model = load_models_and_create_ensemble(s3, model_paths)

    # Save and register the ensemble model in MLflow
    save_and_register_model(ensemble_model, model_name)

In [8]:
if TRAIN:
    if not os.path.exists(pm.path_dataset_processed) or OVERWRITE_PROCESSED_DATA:
        df_train_raw = pd.read_csv(pm.path_data_train_raw)

        if DEBUG:
            df_train_raw = df_train_raw[df_train_raw["stock_id"].isin([0, 1, 2])]

        drop_idx = df_train_raw.loc[
            df_train_raw["target"].isna(), "target"
        ].index.to_list()
        df_train = df_train_raw.drop(drop_idx, axis=0)
        df_train.reset_index(drop=True, inplace=True)
    else:
        df_train = pd.read_csv(pm.path_dataset_processed)
        if DEBUG:
            df_train = df_train[df_train["stock_id"].isin([0, 1, 2])]

In [9]:
if TRAIN:
    global_stock_id_feats = global_stock_id_feats(df_train)

In [10]:

def aggregate_feature_importance(list_files_feat_importance):
    list_of_dfs = []
    for file_path in list_files_feat_importance:
        feature_importance_df = pd.read_csv(file_path)

        folds = [col for col in feature_importance_df.columns if "imp_fold" in col]

        # Normalize by dividing each score by the sum of scores within its respective fold
        for fold in folds:
            fold_sum = feature_importance_df[fold].sum()
            feature_importance_df[fold] = feature_importance_df[fold] / fold_sum

        list_of_dfs.append(feature_importance_df)

    aggregated_df = pd.concat(list_of_dfs, ignore_index=True)

    df_median_importance = aggregated_df.groupby("feat").median().reset_index()

    df_median_importance["feat_imp_overall_mean"] = df_median_importance.loc[
        :, df_median_importance.columns != "feat"
    ].median(axis=1, skipna=True)
    cols = ["feat", "feat_imp_overall_mean"] + [
        col
        for col in df_median_importance.columns
        if col not in ["feat_imp_overall_mean", "feat"]
    ]
    df_median_importance = df_median_importance[cols]

    df_median_importance.sort_values(
        "feat_imp_overall_mean", ascending=False, inplace=True
    )
    return df_median_importance

In [11]:
#aggregate_feature_importance(['/kaggle/working/feat_impor_optiver_trading_at_the_close_23_11_13_2032.csv']).tail(20)

In [12]:
# df_experiments = get_experiments_df(client)

# experiment_ids_to_remove = [
#     "999333986568643837",
#     "867182202959923683",
#     "773486292991054569",
#     "195291970476306379",
# ]
# delete_runs_and_artifacts(client, experiment_ids_to_remove, s3_bucket_name)

In [13]:

@njit(fastmath=True)
def rolling_average(arr, window):
    """
    Calculate the rolling average for a 1D numpy array.
    
    Parameters:
    arr (numpy.ndarray): Input array to calculate the rolling average.
    window (int): The number of elements to consider for the moving average.
    
    Returns:
    numpy.ndarray: Array containing the rolling average values.
    """
    n = len(arr)
    result = np.empty(n)
    result[:window] = np.nan  # Padding with NaN for elements where the window is not full
    cumsum = np.cumsum(arr)

    for i in range(window, n):
        result[i] = (cumsum[i] - cumsum[i - window]) / window

    return result

@njit(parallel=True)
def compute_rolling_averages(df_values, window_sizes):
    """
    Calculate the rolling averages for multiple window sizes in parallel.
    
    Parameters:
    df_values (numpy.ndarray): 2D array of values to calculate the rolling averages.
    window_sizes (List[int]): List of window sizes for the rolling averages.
    
    Returns:
    numpy.ndarray: A 3D array containing the rolling averages for each window size.
    """
    num_rows, num_features = df_values.shape
    num_windows = len(window_sizes)
    rolling_features = np.empty((num_rows, num_features, num_windows))

    for feature_idx in prange(num_features):
        for window_idx, window in enumerate(window_sizes):
            rolling_features[:, feature_idx, window_idx] = rolling_average(df_values[:, feature_idx], window)

    return rolling_features

In [14]:
%%time

@njit(parallel = True)
def calculate_rsi(prices, period=14):
    rsi_values = np.zeros_like(prices)

    for col in prange(prices.shape[1]):
        price_data = prices[:, col]
        delta = np.zeros_like(price_data)
        delta[1:] = price_data[1:] - price_data[:-1]
        gain = np.where(delta > 0, delta, 0)
        loss = np.where(delta < 0, -delta, 0)

        avg_gain = np.mean(gain[:period])
        avg_loss = np.mean(loss[:period])
        
        if avg_loss != 0:
            rs = avg_gain / avg_loss
        else:
            rs = 1e-9  # or any other appropriate default value
            
        rsi_values[:period, col] = 100 - (100 / (1 + rs))

        for i in prange(period-1, len(price_data)-1):
            avg_gain = (avg_gain * (period - 1) + gain[i]) / period
            avg_loss = (avg_loss * (period - 1) + loss[i]) / period
            if avg_loss != 0:
                rs = avg_gain / avg_loss
            else:
                rs = 1e-9  # or any other appropriate default value
            rsi_values[i+1, col] = 100 - (100 / (1 + rs))

    return rsi_values

CPU times: user 359 µs, sys: 22 µs, total: 381 µs
Wall time: 304 µs


In [15]:
def generate_rsi(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for stock_id, values in df.groupby(['stock_id'])[prices]:
        columns = [f'rsi_{col}' for col in values.columns]
        data = calculate_rsi(values.values)
        df.loc[values.index, columns] = data
    
    return df

In [16]:
def generate_macd(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for stock_id, values in df.groupby(['stock_id'])[prices]:
        macd_values, signal_line_values, histogram_values = calculate_macd(values.values)
        col_macd = [f'macd_{col}' for col in values.columns]
        col_signal = [f'macd_sig_{col}' for col in values.columns]
        col_hist = [f'macd_hist_{col}' for col in values.columns]
        
        df.loc[values.index, col_macd] = macd_values
        df.loc[values.index, col_signal] = signal_line_values
        df.loc[values.index, col_hist] = histogram_values
    
    return df

In [17]:
@njit(parallel=True)
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    rows, cols = data.shape
    macd_values = np.empty((rows, cols))
    signal_line_values = np.empty((rows, cols))
    histogram_values = np.empty((rows, cols))

    for i in prange(cols):
        short_ema = np.zeros(rows)
        long_ema = np.zeros(rows)

        for j in range(1, rows):
            short_ema[j] = (data[j, i] - short_ema[j - 1]) * (2 / (short_window + 1)) + short_ema[j - 1]
            long_ema[j] = (data[j, i] - long_ema[j - 1]) * (2 / (long_window + 1)) + long_ema[j - 1]

        macd_values[:, i] = short_ema - long_ema

        signal_line = np.zeros(rows)
        for j in range(1, rows):
            signal_line[j] = (macd_values[j, i] - signal_line[j - 1]) * (2 / (signal_window + 1)) + signal_line[j - 1]

        signal_line_values[:, i] = signal_line
        histogram_values[:, i] = macd_values[:, i] - signal_line

    return macd_values, signal_line_values, histogram_values

In [18]:
# generate imbalance features
def imbalance_features(df):
    prices = [
        "reference_price",
        "far_price",
        "near_price",
        "ask_price",
        "bid_price",
        "wap",
        "imbalance_size"
    ]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval(
        "(imbalance_size-matched_size)/(matched_size+imbalance_size)"
    )
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [["ask_price", "bid_price", "wap", "reference_price"], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values

    # V2
    weights = convert_weights_to_dict()
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df["wap_momentum"] = df.groupby("stock_id")["weighted_wap"].pct_change(periods=6)
    df["imbalance_momentum"] = (
        df.groupby(["stock_id"])["imbalance_size"].diff(periods=1) / df["matched_size"]
    )
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(["stock_id"])["price_spread"].diff()
    df["price_pressure"] = df["imbalance_size"] * (df["ask_price"] - df["bid_price"])
    df["market_urgency"] = df["price_spread"] * df["liquidity_imbalance"]
    df["depth_pressure"] = (df["ask_size"] - df["bid_size"]) * (
        df["far_price"] - df["near_price"]
    )
    df["spread_depth_ratio"] = (df["ask_price"] - df["bid_price"]) / (
        df["bid_size"] + df["ask_size"]
    )
    df["mid_price_movement"] = (
        df["mid_price"]
        .diff(periods=5)
        .apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    )
    df["micro_price"] = (
        (df["bid_price"] * df["ask_size"]) + (df["ask_price"] * df["bid_size"])
    ) / (df["bid_size"] + df["ask_size"])
    df["relative_spread"] = (df["ask_price"] - df["bid_price"]) / df["wap"]

    
    # V4 features - Rolling averages
    window_sizes = [6]  # Define your desired window sizes
    list_cols = ["wap","imbalance_size"]
    for price in prices:
        rolling_avg_features = compute_rolling_averages(df[price].values.reshape(-1, 1), window_sizes)

        # Assigning the rolling average results to the DataFrame
        for i, window in enumerate(window_sizes):
            column_name = f"{price}_rolling_avg_{window}"
            df[column_name] = rolling_avg_features[:, 0, i]

    # Patching the start-of-day values after all rolling averages are calculated
    for window in window_sizes:
        for idx, seconds in enumerate(df['seconds_in_bucket'].values):
            if seconds / 10 <= window:
                for price in prices:
                    column_name = f"{price}_rolling_avg_{window}"
                    df.at[idx, column_name] = df.at[idx, price]
                    
                    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)

    # V3
    for col in [
        "matched_size",
        "imbalance_size",
        "reference_price",
        "imbalance_buy_sell_flag"
    ]:
        for window in [1, 2, 3, 5, 10]:

            df[f"{col}_shift_{window}"] = df.groupby(["stock_id"])[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby(["stock_id"])[col].pct_change(window)

    for col in [
        "ask_price",
        "bid_price",
        "ask_size",
        "bid_size",
        "wap",
        "near_price",
        "far_price",
        "market_urgency"
    ]:
        for window in [1, 2, 3, 5, 10]:
            df[f"{col}_diff_{window}"] = df.groupby(["stock_id"])[col].diff(window)

    return df.replace([np.inf, -np.inf], 0)


# generate time & stock features
def other_features(df):
    df["dow"] = df["date_id"] % 5
    df["dom"] = df["date_id"] % 20
    df["seconds"] = df["seconds_in_bucket"] % 60
    df["minute"] = df["seconds_in_bucket"] // 60

    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df


# generate all features
def feat_engineering(df):
    df.reset_index(inplace = True, drop = True)
    cols = [c for c in df.columns if c not in ["row_id"]]
    df = df[cols]
    df = imbalance_features(df)
    df = other_features(df)
    
        
    df = generate_rsi(df)
    df = generate_macd(df)
    gc.collect()

    list_cols = [i for i in df.columns if i not in ["row_id"] + ['depth_pressure',
         'bid_size_diff_5',
         'ask_size_diff_5',
         'ask_size_diff_10',
         'imbalance_buy_sell_flag_ret_5',
         'imbalance_buy_sell_flag_ret_10',
         'imbalance_buy_sell_flag_ret_3',
         'imbalance_buy_sell_flag_ret_2',
         'minute','bid_size_diff_3',
         'bid_size_diff_10',
         'ask_price_diff_10',
         'bid_price_diff_10',
         'macd_hist_far_price',
         'macd_far_price',
         'macd_sig_near_price',
         'macd_sig_far_price',
         'macd_near_price',
         'macd_hist_near_price','bid_size_diff_2','volume','wap_diff_10','imbalance_buy_sell_flag_ret_1','bid_size_ask_size_imbalance_size_imb2','ask_size_diff_2']]

    
    

    return df[list_cols]

In [19]:
if TRAIN:
    df_train_feats = feat_engineering(df_train)
    print("Build Online Train Feats Finished.")

    df_train_feats = reduce_mem_usage(df_train_feats)

In [20]:
if TRAIN:
    col_split = "time_id"
    df_train_feats.sort_values([col_split], inplace=True)
    df_train_feats.reset_index(drop=True, inplace=True)
    df_train_feats["factorized"] = pd.factorize(df_train_feats[col_split])[0]

    list_cols_drop = ["date_id", "time_id"]
    df_train_feats.drop(list_cols_drop, axis=1, inplace=True)

In [21]:
if TRAIN:
    experiment_name = f"{EXPERIMENT_PURPOSE}_v{VERSION_NB}"
    name_folder_models = f"models_v{VERSION_NB}"

    experiment_date_str = datetime.now().strftime("%y_%m_%d_%H%M")
    experiment_id = get_or_create_experiment(
        client, experiment_name, artifact_location=pm.path_artifact_location
    )


    

    nbrnd_erly_stp = 130
    cv_mthd = "KF"

    # Cross-Validation Setup
    if TRAIN:
        # Initialize MLflow callback
        mlflow_callback = MLflowCallback(
            tracking_uri=mlflow.get_tracking_uri(), metric_name="mae"
        )

        all_cv = {"KF": KFold(n_splits=5, shuffle=True, random_state=STATE)}
        cv = all_cv[cv_mthd]

        model_params_dict = {
            "LGBMR": {
                "static_params": {
                    "device": "gpu" if gpu_switch == "ON" else "cpu",
                    "objective": "mae",
                    "boosting_type": "gbdt",
                    "random_state": STATE,
                    "n_jobs": 4,
                    "verbose": -1,
                    "importance_type": "gain",
                },
                "dynamic_params": {
                    "n_estimators": {
                        "type": "int",
                        "low": 4052,
                        "high": 4052,
                    },
                    "learning_rate": {
                        "type": "float",
                        "low": 0.012,
                        "high": 0.012,
                    },
                    "max_depth": {"type": "int", "low": 49, "high": 49},
                    "num_leaves": {
                        "type": "int",
                        "low": 63,
                        "high": 63,
                    },
                    "min_child_samples": {
                        "type": "int",
                        "low": 23,
                        "high": 23,
                    },
                    "subsample": {
                        "type": "float",
                        "low": 1,
                        "high": 1,
                    },
                    "colsample_bytree": {
                        "type": "float",
                        "low": 1,
                        "high": 1,
                    },
                  
                   
                },
            },
        }

        dict_models = {"LGBMR": LGBMR}

        log_model = True


    if TRAIN:
        args = {
            "cv_mthd": cv_mthd,
            "experiment_purpose": EXPERIMENT_PURPOSE,
            "experiment_name": experiment_name,
            "dict_models": dict_models,
            "model_params_dict": model_params_dict,
            "n_splits": N_SPLITS,
            "n_test_splits": N_TEST_SPLITS,
            "n_purge": N_PURGE,
            "n_embargo": N_EMBARGO,
            "experiment_date_str": experiment_date_str,
            "path_artifact_location": pm.path_artifact_location,
            "target_col": "target",
        }

In [22]:
def run_mlflow_experiment(df_train, args, trial=None):
    cv_mthd = args["cv_mthd"]
    experiment_purpose = args["experiment_purpose"]
    experiment_name = args["experiment_name"]
    dict_models = args["dict_models"]
    model_params_dict = args["model_params_dict"]

    n_splits = args["n_splits"]
    n_test_splits = args["n_test_splits"]
    n_purge = args["n_purge"]
    n_embargo = args["n_embargo"]

    experiment_date_str = args["experiment_date_str"]
    path_artifact_location = args["path_artifact_location"]
    target_col = args["target_col"]

    if trial == None:
        trial = optuna.trial.FixedTrial(
            {
                "n_estimators": 500,
                "learning_rate": 0.005,
                "max_depth": 10,
                "num_leaves": 20,
                "min_child_samples": 10,
                "subsample": 0.7,
                "colsample_bytree": 1.0,
                "min_split_gain": 0.0,
                "reg_alpha": 0.0,
                "reg_lambda": 0.0,
                "device": "gpu" if gpu_switch == "ON" else "cpu",
            }
        )

    run_time_start_trial = datetime.now().strftime("%y_%m_%d_%H%M%S")

    with mlflow.start_run(
        run_name=run_time_start_trial, experiment_id=experiment_id
    ) as run:
        score_list = []

        # mlflow.set_tag("cv_mthd", cv_mthd)
        mlflow.set_tag("n_splits", n_splits)
        mlflow.set_tag("n_test_splits", n_test_splits)
        mlflow.set_tag("n_purge", n_purge)
        mlflow.set_tag("n_embargo", n_embargo)

        for model_name, model_class in dict_models.items():
            if TUNING:
                model = create_model(
                    trial,
                    model_class,
                    model_params_dict[model_name]["static_params"],
                    model_params_dict[model_name]["dynamic_params"],
                )
            else:
                params = model_prod.get_params()
                params['device'] = "gpu"
                
                model = model_class(**params)

            priority_params = ["learning_rate", "max_depth"]
            excluded_params = [
                "device",
                "class_weight",
                "random_state",
                "silent",
                "verbose",
                "n_jobs",
            ]

            ordered_params = log_model_parameters(
                model, priority_params, excluded_params, verbose=True
            )

            mlflow.log_params(ordered_params)

            for fold_n, (train_indices, test_indices) in enumerate(
                time_series_split(
                    df_train,
                    n_splits=n_splits,
                    n_test_splits=n_test_splits,
                    n_purge=n_purge,
                    n_embargo=n_embargo,
                )
            ):
               # if fold_n == 0:
                with mlflow.start_run(
                    run_name=f"fold_{fold_n+1}",
                    nested=True,
                    experiment_id=experiment_id,
                ) as nested_run:
                    mlflow.set_tag("n_trial", str(trial.number))

                    mask_train = df_train["factorized"].isin(train_indices)
                    mask_test = df_train["factorized"].isin(test_indices)

                    y_train = df_train.loc[mask_train, target_col]
                    y_val = df_train.loc[mask_test, target_col]
                    X_train = df_train.loc[mask_train].drop(
                        [target_col, "factorized"], axis=1
                    )
                    X_val = df_train.loc[mask_test].drop(
                        [target_col, "factorized"], axis=1
                    )

                    mlflow.log_param("train_rows", X_train.shape[0])
                    mlflow.log_param("train_cols", X_train.shape[1])

                    model.fit(
                        X_train,
                        y_train,
                        eval_set=[(X_val, y_val)],
                        eval_metric="mae",
                        callbacks=[
                            lgbm.callback.early_stopping(stopping_rounds=100),
                            lgbm.callback.log_evaluation(period=100000),
                        ],
                    )

                    log_feature_importance(
                        trial.number,
                        model,
                        X_train,
                        fold_n,
                        experiment_purpose,
                        experiment_date_str,
                    )

                    fold_score = model.best_score_["valid_0"]["l1"]

                    score_list.append(fold_score)

                    mlflow.log_metric("fold_score", round(fold_score, 6))
                    mlflow.log_param("fold_number", fold_n + 1)
                    mlflow.log_param("model_name", model_name)

                    mlflow.log_params(ordered_params)

                    current_time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
                    model_log_name = f"{model_name}_{trial.number}_{current_time_str}"

                    mlflow.sklearn.log_model(model, model_log_name)

                    mlflow.log_param("run_time", current_time_str)

                    nested_run_id = nested_run.info.run_id
                    model_path = f"{path_artifact_location}/{run.info.experiment_id}/{nested_run_id}/artifacts/{model_log_name}/model.pkl"
                    mlflow.log_param("model_path", model_path)

                avg_score = sum(score_list) / len(score_list)
                median_score = np.median(score_list)
                mlflow.log_metric("avg score", round(avg_score, 6))
                mlflow.log_metric("median score", round(median_score, 6))

        return avg_score


def objective(trial, df_train):
    avg_score = run_mlflow_experiment(df_train, args, trial)
    return avg_score


# Run the Optuna study
if TRAIN:
    study = optuna.create_study(
        direction="minimize",
        study_name="Your Study Name",
        load_if_exists=True,
    )
    study.optimize(lambda trial: objective(trial, df_train_feats), n_trials=N_TRIALS)

In [23]:
if TRAIN:
    df_exp = experiments_data(
        client, list_experiment_id=None, save_df=None, list_columns=None
    )
    list_base_cols = [
        "run_time",
        "experiment_id",
        "n_trial",
        "run_id",
        "model_name",
        "fold_number",
        "fold_score",
    ]
    list_dynamic_params = list(model_params_dict["LGBMR"]["dynamic_params"].keys())

    df_exp["run_time"] = pd.to_datetime(
        df_exp["run_time"], format="%Y%m%d_%H%M%S", errors="coerce"
    )

    for col in df_exp.columns:
        df_exp[col] = pd.to_numeric(df_exp[col], errors="ignore")

    for col in df_exp.select_dtypes(include=["float", "int"]):
        df_exp[col] = df_exp[col].round(5)

    list_cols_exp = ["run_time"] + list_base_cols + list_dynamic_params + ["model_path"]

    experiment_id
    df_exp = df_exp[df_exp["experiment_id"] != 0]

    df_exp = df_exp[list_cols_exp]

In [24]:
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()

In [25]:
df_train = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")


In [26]:
global_stock_id_feats = global_stock_id_feats(df_train)

In [27]:
counter = 0
df_tot_test = []
for test, revealed_targets, sample_prediction in iter_test:
    test["time_id"] = counter
    test["target"] = "none"

    if counter < 17:
        df_tot_test.append(test)
    else:
        df_tot_test = df_tot_test[1:]
        df_tot_test.append(test)

    df_test = pd.concat(df_tot_test, axis=0, ignore_index=True)

    feat = feat_engineering(df_test)
    feat = feat.sort_values(["date_id", "seconds_in_bucket", "stock_id"])[-len(test) :]

    list_cols_drop = ["date_id"]
    feat.drop(list_cols_drop, axis=1, inplace=True)

    list_features = model_prod.models[0].feature_name_
    feat = feat[list_features]
    sample_prediction["target"] = model_prod.predict(feat, 'mean')
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
